In [ ]:
from sklearn.model_selection import train_test_split
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

In [ ]:
# Assuming the dataset is in a CSV file
df = pd.read_csv("LIAR2/train.csv", delimiter=',')
statements = df['statement'].dropna().tolist() 
subjects = df['subject'].dropna().tolist()  

In [ ]:


# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(statements, subjects, test_size=0.2, random_state=42)

# Initialize a tokenizer and model for embeddings (using a pre-trained BERT model)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Function to encode text using BERT embeddings
def encode_text(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).logits
    return embeddings.numpy()

# Use the above function to encode the training and test texts
X_train_embeddings = encode_text(X_train)
X_test_embeddings = encode_text(X_test)

# Initialize BERTopic with supervision (use the subjects as labels)
topic_model = BERTopic(language="english")
topic_model.fit(X_train_embeddings, y_train)

# Evaluate the model
y_pred = topic_model.transform(X_test_embeddings)
print(classification_report(y_test, y_pred))
